In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
df_AFKS = pd.read_csv("AFKS.csv")
data_AFKS = pd.DataFrame(df_AFKS)
data_AFKS['Цена'] = data_AFKS['Цена'].str.replace(',','.')
data_AFKS = data_AFKS[:201]

prices_AFKS = []
for price in data_AFKS['Цена']:
    prices_AFKS.append(price)

In [3]:
df_NVTK = pd.read_csv("NVTK.csv")
data_NVTK = pd.DataFrame(df_NVTK)
data_NVTK['Цена'] = data_NVTK['Цена'].str.replace(',','.')
data_NVTK = data_NVTK[:201]

prices_NVTK = []
for price in data_NVTK['Цена']:
    prices_NVTK.append(price)

In [4]:
df_SNGS = pd.read_csv("SNGS.csv")
data_SNGS = pd.DataFrame(df_SNGS)
data_SNGS['Цена'] = data_SNGS['Цена'].str.replace(',','.')
data_SNGS = data_SNGS[:201]

prices_SNGS = []
for price in data_SNGS['Цена']:
    prices_SNGS.append(price)

In [5]:
#Преобразование данных
for i in range(len(prices_AFKS)):
    prices_AFKS[i] = float(prices_AFKS[i])
    
for i in range(len(prices_NVTK)):
    prices_NVTK[i] = float(prices_NVTK[i])
    
for i in range(len(prices_SNGS)):
    prices_SNGS[i] = float(prices_SNGS[i])
    
print("prices_AFKS:", prices_AFKS[:5])
print("prices_NVTK:", prices_NVTK[:5])
print("prices_SNGS:", prices_SNGS[:5])

prices_AFKS: [17.65, 17.7, 17.34, 17.59, 17.61]
prices_NVTK: [591.0, 599.5, 579.2, 581.8, 587.7]
prices_SNGS: [33.975, 34.55, 33.675, 33.915, 33.945]


In [6]:
#Нетто-доходность
netto_AFKS = [0] * 200
netto_NVTK = [0] * 200
netto_SNGS = [0] * 200
for i in range(1, 201):
    netto_AFKS[i-1] = prices_AFKS[i] / prices_AFKS[i-1] - 1
    netto_NVTK[i-1] = prices_NVTK[i] / prices_NVTK[i-1] - 1
    netto_SNGS[i-1] = prices_SNGS[i] / prices_SNGS[i-1] - 1

print(len(netto_AFKS), len(netto_NVTK), len(netto_SNGS))
print("netto_AFKS:", netto_AFKS[:5])
print("netto_NVTK:", netto_NVTK[:5])
print("netto_SNGS:", netto_SNGS[:5])

200 200 200
netto_AFKS: [0.0028328611898016387, -0.02033898305084747, 0.014417531718569743, 0.0011370096645821892, 0.005110732538330387]
netto_NVTK: [0.014382402707275865, -0.03386155129274393, 0.0044889502762428535, 0.010140941904434708, 0.005615109749872227]
netto_SNGS: [0.016924208977189048, -0.02532561505065123, 0.007126948775055819, 0.0008845643520567492, 0.0016202680807189118]


In [7]:
#Среднее значение нетто-доходности
mean_AFKS = np.mean(netto_AFKS)
mean_NVTK = np.mean(netto_NVTK)
mean_SNGS = np.mean(netto_SNGS)

print("mean_AFKS =", mean_AFKS)
print("mean_NVTK =", mean_NVTK)
print("mean_SNGS =", mean_SNGS)

mean_AFKS = -0.00017457040217465412
mean_NVTK = -0.0010634269129275104
mean_SNGS = 0.00020605427021471112


In [8]:
#Дисперсия
dispersion_AFKS = np.mean((netto_AFKS - mean_AFKS)**2)
dispersion_NVTK = np.mean((netto_NVTK - mean_NVTK)**2)
dispersion_SNGS = np.mean((netto_SNGS - mean_SNGS)**2)

print("dispersion_AFKS =", dispersion_AFKS)
print("dispersion_NVTK =", dispersion_NVTK)
print("dispersion_SNGS =", dispersion_SNGS)

dispersion_AFKS = 0.00031399636089890633
dispersion_NVTK = 0.00029100734338162687
dispersion_SNGS = 0.000373327876803058


In [9]:
#Матрица ковариаций
X = np.array([netto_AFKS, netto_NVTK, netto_SNGS])
covariation = np.cov(X)
print(covariation)

[[0.00031557 0.00010769 0.00013394]
 [0.00010769 0.00029247 0.00012679]
 [0.00013394 0.00012679 0.0003752 ]]


# Составим задачу минимизации дисперсии портфеля

D($\sum_{i=1}^{3} x_{i} r_{i}$) = D($x_{1} r_{1}$ + $x_{2} r_{2}$ + $x_{3} r_{3}$) = $\sum_{i=1}^{3} x_{i}^{2} D(r_{i})$ + 2 $x_{1} x_{2}$ cov($r_{1}$, $r_{2}$) + 2 $x_{2} x_{3}$ cov($r_{2}$, $r_{3}$) + 2 $x_{1} x_{3}$ cov($r_{1}$, $r_{3}$)

### Составим систему
\begin{equation*}
    \begin{cases}
      f0(x_{1}, x_{2}, x_{3}) = D_{11} * x_{1}^{2} + D_{22} * x_{2}^{2} + D_{33} * x_{3}^{2} + 2 * D_{12} * x_{1} * x_{2} + 2 * D_{23} * x_{2} * x_{3} + 2 * D_{13} * x_{1} * x_{3} \ \rightarrow min \\
      f1(x_{1}, x_{2}, x_{3}) = \mu_1 * x_{1} + \mu_2 * x_{2} + \mu_3 * x_{3} \ = \mu* \\
      f2(x_{1}, x_{2}, x_{3}) = x_{1} + x_{2} + x_{3} = 1 \\
      -0.0010634269129275104 < \mu* < 0.00020605427021471112 \\
      x_{1} \geqslant 0 \\
      x_{2} \geqslant 0 \\
      x_{3} \geqslant 0
    \end{cases}
\end{equation*}

### Решение системы

Возмем $\mu*$ = 0.00001

Составим функцию Лагранжа.

$$L(x_{1}, x_{2}, x_{3}) = \lambda_{0} f_{0}(x_{1}, x_{2}, x_{3}) + \lambda_{1} (f_{1}(x_{1}, x_{2}, x_{3}) - 0.00001) + \lambda_{2} (f_{2}(x_{1}, x_{2}, x_{3}) - 1)$$

Нет условий дополняющей нежесткости, так как нет неравенств.

Составим систему.

\begin{equation*}
    \begin{cases}
      \sigma_{x1} L(x_{1}, x_{2}, x_{3}) = 0 \\
      \sigma_{x2} L(x_{1}, x_{2}, x_{3}) = 0 \\
      \sigma_{x3} L(x_{1}, x_{2}, x_{3}) = 0 \\
      f1(x_{1}, x_{2}, x_{3}) - 0.00001 = 0 \\
      f2(x_{1}, x_{2}, x_{3}) - 1 = 0 \\
      x_{1} \geqslant 0 \\
      x_{2} \geqslant 0 \\
      x_{3} \geqslant 0
    \end{cases}
\end{equation*}

In [17]:
from sympy import *

#Вводим переменные
x1 = Symbol('x1')
x2 = Symbol('x2')
x3 = Symbol('x3')
lambda1 = Symbol('l1')
lambda2 = Symbol('l2')

#обозначим ковариацию
D11 = covariation[0][0]
D12 = covariation[0][1]
D13 = covariation[0][2]
D21 = covariation[1][0]
D22 = covariation[1][1]
D23 = covariation[1][2]
D31 = covariation[2][0]
D32 = covariation[2][1]
D33 = covariation[2][2]

#обозначим среднюю
m1 = mean_AFKS
m2 = mean_NVTK
m3 = mean_SNGS

#Уравнения
f0 = D11 * x1**2 + D22 * x2**2 + D33 * x3**2 + 2*D12*x1*x2 + 2*D13*x1*x3 + 2*D23*x3*x2
f1 = m1 * x1 + m2 * x2 + m3 * x3
f2 = x1 + x2 + x3

#### Решим систему
1) $\lambda_{0}$ = 0

In [24]:
lagrange_function_0 = lambda1 * (f1 - 0.00001) + lambda2 * (f2 - 1)

nonlinsolve([
    lagrange_function_0.diff(x1),
    lagrange_function_0.diff(x2),
    lagrange_function_0.diff(x3),
    f1 - 0.00001,
    f2 - 1,
    x1 >= 0,
    x2 >= 0,
    x3 >= 0
], [x1, x2, x3, lambda1, lambda2])

{(1.2076492661547 - 1.42821835446418*x3, 0.428218354464185*x3 - 0.207649266154696, x3, 0, 0)}

Вектор $\lambda$ = ($\lambda_{0}$, $\lambda_{1}$, $\lambda_{2}$) получился нулевой.
Следовательно, $\lambda_{0} \ne 0$ 

2) $\lambda_{0} = 1$

In [25]:
lagrange_function_1 = f0 + lambda1 * (f1 - 0.00001) + lambda2 * (f2 - 1)

nonlinsolve([
    lagrange_function_1.diff(x1),
    lagrange_function_1.diff(x2),
    lagrange_function_1.diff(x3),
    f1 - 0.00001,
    f2 - 1,
    x1 >= 0,
    x2 >= 0,
    x3 >= 0
], [x1, x2, x3, lambda1, lambda2])

{(0.46388843901588, 0.015350274858619, 0.520761286125501, -0.218987536925042, -0.00047381993391644)}

#### Ответ:
Получили решение системы.

Вектор $x = (x_{1}, x_{2}, x_{3})$ удовлетворяет множеству допустимых решений $(x_{1} \geqslant 0, x_{2} \geqslant 0, x_{3} \geqslant 0)$

Следовательно, по теореме Куна-Таккера полученный вектор х является решением системы.

$x = (0.46388843901588, 0.015350274858619, 0.520761286125501)$

### Работу выполнил Эшкинин Андрей Игоревич БПМ-203